In [2]:
import dexplor_utils
import prep2_utils
%matplotlib inline

In [3]:
all_train, spray, weather, test, enrich = prep2_utils.load_data()


name: train, rows: 10506, columns: 12, null values: 0

name: spray, rows: 14835, columns: 4, null values: 584

name: weather, rows: 2944, columns: 22, null values: 0

name: test, rows: 116293, columns: 11, null values: 0

name: enrich, rows: 27196, columns: 13, null values: 12324

name: train, date range: 2007-05-29 - 2013-09-26

name: spray, date range: 2011-08-29 - 2013-09-05

name: weather, date range: 2007-05-01 - 2014-10-31

name: test, date range: 2008-06-11 - 2014-10-02

name: enrich, date range: 05/28/2009 12:05:00 AM - 10/09/2007 12:10:00 AM


#### Save labels

In [ ]:
y_train = all_train[['WnvPresent']]
y_train.to_csv('y_train.csv', columns=y_train.columns.tolist())

#### preprocess_dataset

1. get relevant columns
2. add distance from each weather station
3. add weather features (weighted avg by dist from each station)
4. add dummies for positive 3 strains in train set
5. convert traps to integers
6. add duplicate row flag 
7. add date features (day, week, month, day of week)
8. drop irrelevant cols
9. save

#### add weather features and split by station

In [ ]:
weather = weather[prep2_utils.weather_cols]
weather_stat1, weather_stat2 = prep2_utils.split_weather_by_station(weather)

In [ ]:
wnv_species = all_train[all_train['WnvPresent'] == 1].Species.unique()
all_traps = test.Trap.unique().tolist()

In [ ]:
x_train = prep2_utils.preprocess_dataset(
    all_train, 'x_train3', weather_stat1, weather_stat2, all_traps, wnv_species, spray)

x_test = prep2_utils.preprocess_dataset(
    test, 'x_test3', weather_stat1, weather_stat2, all_traps, wnv_species, spray)

In [ ]:
all_train.columns.tolist() == test.columns.tolist()

In [ ]:
wnv_by_year = all_train[['Date', 'Trap', 'WnvPresent']].groupby(
        by = ['Date','Trap'])['Date','Trap','WnvPresent'].sum().reset_index().sort_values(
        'WnvPresent', ascending = False).set_index('Date')

In [ ]:
plot_train = x_train.merge(all_train[['Date']], how = 'left', right_index = True, left_index = True)
prep2_utils.plot_weather_features(plot_train, prep2_utils.weather_features, wnv_by_year)

In [ ]:
test_wnv_by_year = enrich[['Date', 'Trap', 'WnvPresent']].groupby(
        by = ['Date','Trap'])['Date','Trap','WnvPresent'].sum().reset_index().sort_values(
        'WnvPresent', ascending = False).set_index('Date')

In [ ]:
plot_test = x_test.merge(test[['Date']], how = 'left', right_index = True, left_index = True)
prep2_utils.plot_weather_features(plot_test, prep2_utils.weather_features, test_wnv_by_year)

In [ ]:
dexplor_utils.plot_weather_features(
    plot_train, ['Tavg', 'PrecipTotal', 'AvgSpeed', 'day_count', 'day_of_week', 'week' ,'month'])

In [ ]:
dexplor_utils.plot_weather_features(
    plot_test, ['Tavg', 'PrecipTotal', 'AvgSpeed', 'day_count', 'day_of_week', 'week' ,'month'])

In [ ]:
weather.shape, spray.shape, x_train.shape, test.shape, y_train.shape

In [ ]:
test.head()